# ETL NBA Limpieza - Proyecto Daft17_Group01_PF (Versión 2)
Este notebook procesa los datasets crudos de la NBA, normalizando nombres, fechas, años, nulos, duplicados semánticos y ahora corrigiendo altura y peso.


In [3]:

# ==========================================================
# BLOQUE 1: IMPORTACIÓN Y CONFIGURACIÓN INICIAL
# ==========================================================
import os
import re
import unicodedata
import pandas as pd
import numpy as np
from datetime import datetime

path_raw = r"C:\Users\juanl\Downloads\final\csv\Daft17_Group01_PF\data\raw"
path_final = r"C:\Users\juanl\Downloads\final\csv"

input_files = {
    "game": "game.csv",
    "game_summary": "game_summary.csv",
    "other_stats": "other_stats.csv",
    "player": "player.csv",
    "common_player_info": "common_player_info.csv",
    "team": "team.csv"
}

dataframes = {}
for name, file in input_files.items():
    try:
        full_path = os.path.join(path_raw, file)
        df = pd.read_csv(full_path, low_memory=False)
        dataframes[name] = df
        print(f"✅ {file} cargado correctamente ({df.shape[0]} filas, {df.shape[1]} columnas)")
    except Exception as e:
        print(f"❌ Error cargando {file}: {e}")

# ==========================================================
# BLOQUE 2: NORMALIZACIÓN DE NOMBRES DE COLUMNAS
# ==========================================================
def limpiar_nombre_columna(nombre):
    nombre = unicodedata.normalize("NFKC", str(nombre))
    nombre = nombre.strip().lower()
    nombre = re.sub(r"[áàäâ]", "a", nombre)
    nombre = re.sub(r"[éèëê]", "e", nombre)
    nombre = re.sub(r"[íìïî]", "i", nombre)
    nombre = re.sub(r"[óòöô]", "o", nombre)
    nombre = re.sub(r"[úùüû]", "u", nombre)
    nombre = re.sub(r"[^a-z0-9]+", "_", nombre)
    return nombre.strip("_")

def normalizar_columnas(df):
    df.columns = [limpiar_nombre_columna(c) for c in df.columns]
    return df

for name, df in dataframes.items():
    dataframes[name] = normalizar_columnas(df)
    print(f"📘 Columnas normalizadas en {name}: {len(df.columns)} columnas")

# ==========================================================
# BLOQUE 3: CORRECCIÓN DE FECHAS Y AÑOS
# ==========================================================
def truncar_anio(valor):
    try:
        val = str(int(float(valor)))
        if len(val) > 4:
            return int(val[:4])
        return int(val)
    except:
        return np.nan

def corregir_anios_y_fechas(df):
    for col in df.columns:
        if "year" in col or "anio" in col or "season" in col:
            df[col] = df[col].apply(truncar_anio).astype("Int64")
        if "date" in col or "fecha" in col:
            df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
    return df

for name, df in dataframes.items():
    dataframes[name] = corregir_anios_y_fechas(df)
    print(f"📅 Años y fechas corregidos en {name}")

# ==========================================================
# BLOQUE 3.1: CORRECCIÓN DE ALTURA Y PESO (AJUSTADO)
# ==========================================================
# - Altura: reemplaza "-" por "," en columnas tipo "height" o "altura"
# - Peso: divide por 10 los valores sospechosamente grandes (>400)
# ==========================================================

# ==========================================================
# BLOQUE 3.1: CORRECCIÓN DE ALTURA Y PESO (FINAL)
# ==========================================================
# - Altura: reemplaza "-" por "," en columnas tipo "height" o "altura"
# - Peso: divide SIEMPRE todos los valores por 10
# ==========================================================

def corregir_altura_peso(df):
    for col in df.columns:
        # Altura: reemplazar guiones por comas
        if "height" in col or "altura" in col:
            df[col] = df[col].astype(str).str.replace("-", ",", regex=False)

        # Peso: convertir y dividir todos los valores por 10
        if "weight" in col or "peso" in col:
            df[col] = pd.to_numeric(df[col], errors="coerce") / 10

    return df

for name, df in dataframes.items():
    if any("height" in c or "peso" in c or "weight" in c or "altura" in c for c in df.columns):
        dataframes[name] = corregir_altura_peso(df)
        print(f"🏋️‍♂️ Altura y peso corregidos en {name}")


# ==========================================================
# BLOQUE 4: DETECCIÓN DE COLUMNAS DUPLICADAS O SIMILARES
# ==========================================================
from difflib import SequenceMatcher

def columnas_similares(df, threshold=0.9):
    cols = df.columns
    similares = []
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            ratio = SequenceMatcher(None, cols[i], cols[j]).ratio()
            if ratio >= threshold:
                similares.append((cols[i], cols[j], round(ratio,2)))
    return similares

for name, df in dataframes.items():
    similares = columnas_similares(df)
    if similares:
        print(f"⚠️ Posibles duplicadas en {name}:")
        for c1, c2, r in similares:
            print(f"   - {c1} ≈ {c2} ({r*100:.0f}% similitud)")
    else:
        print(f"✅ {name}: sin columnas duplicadas evidentes.")

# ==========================================================
# BLOQUE 5: LIMPIEZA DE NULOS
# ==========================================================
def limpiar_nulos(df):
    df = df.loc[:, df.isna().mean() < 0.8]
    id_cols = [c for c in df.columns if 'id' in c]
    if id_cols:
        df = df.dropna(subset=id_cols)
    num_cols = df.select_dtypes(include='number').columns
    for col in num_cols:
        if df[col].isna().mean() > 0:
            df[col] = df[col].fillna(df[col].median() if df[col].isna().mean() < 0.3 else 0)
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        df[col] = df[col].fillna('Desconocido')
    return df

for name, df in dataframes.items():
    dataframes[name] = limpiar_nulos(df)
    print(f"🧽 Nulos limpiados en {name}: {df.shape[0]} filas, {df.shape[1]} columnas")

# ==========================================================
# BLOQUE 6: EXPORTACIÓN DE ARCHIVOS LIMPIOS
# ==========================================================
os.makedirs(path_final, exist_ok=True)

for name, df in dataframes.items():
    out_path = os.path.join(path_final, f"{name}_clean.csv")
    df.to_csv(out_path, index=False)
    print(f"✅ {name} guardado en: {out_path}")

# ==========================================================
# BLOQUE 7: REPORTE DE CALIDAD DE DATOS
# ==========================================================
resumen = []
for name, df in dataframes.items():
    resumen.append({
        "Dataset": name,
        "Filas": len(df),
        "Columnas": len(df.columns),
        "Porc_nulos_prom": round(df.isna().mean().mean()*100, 2),
        "Numéricas": len(df.select_dtypes(include='number').columns),
        "Texto": len(df.select_dtypes(include='object').columns),
        "Fechas": sum("datetime" in str(t) for t in df.dtypes)
    })

reporte_df = pd.DataFrame(resumen)
print("\n📋 Resumen general de los datasets:")
display(reporte_df)


✅ game.csv cargado correctamente (65698 filas, 55 columnas)
✅ game_summary.csv cargado correctamente (58110 filas, 14 columnas)
✅ other_stats.csv cargado correctamente (28271 filas, 26 columnas)
✅ player.csv cargado correctamente (4831 filas, 5 columnas)
✅ common_player_info.csv cargado correctamente (4171 filas, 33 columnas)
✅ team.csv cargado correctamente (30 filas, 7 columnas)
📘 Columnas normalizadas en game: 55 columnas
📘 Columnas normalizadas en game_summary: 14 columnas
📘 Columnas normalizadas en other_stats: 26 columnas
📘 Columnas normalizadas en player: 5 columnas
📘 Columnas normalizadas en common_player_info: 33 columnas
📘 Columnas normalizadas en team: 7 columnas


C:\Users\juanl\AppData\Local\Temp\ipykernel_10084\2433312825.py:72: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
C:\Users\juanl\AppData\Local\Temp\ipykernel_10084\2433312825.py:72: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)


📅 Años y fechas corregidos en game
📅 Años y fechas corregidos en game_summary
📅 Años y fechas corregidos en other_stats
📅 Años y fechas corregidos en player
📅 Años y fechas corregidos en common_player_info
📅 Años y fechas corregidos en team
🏋️‍♂️ Altura y peso corregidos en common_player_info
⚠️ Posibles duplicadas en game:
   - fgm_home ≈ fg3m_home (94% similitud)
   - fga_home ≈ fg3a_home (94% similitud)
   - fg_pct_home ≈ fg3_pct_home (96% similitud)
   - fg_pct_home ≈ ft_pct_home (91% similitud)
   - oreb_home ≈ reb_home (94% similitud)
   - dreb_home ≈ reb_home (94% similitud)
   - fgm_away ≈ fg3m_away (94% similitud)
   - fga_away ≈ fg3a_away (94% similitud)
   - fg_pct_away ≈ fg3_pct_away (96% similitud)
   - fg_pct_away ≈ ft_pct_away (91% similitud)
   - oreb_away ≈ reb_away (94% similitud)
   - dreb_away ≈ reb_away (94% similitud)
✅ game_summary: sin columnas duplicadas evidentes.
✅ other_stats: sin columnas duplicadas evidentes.
✅ player: sin columnas duplicadas evidentes.


C:\Users\juanl\AppData\Local\Temp\ipykernel_10084\2433312825.py:72: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)


⚠️ Posibles duplicadas en common_player_info:
   - display_first_last ≈ display_fi_last (91% similitud)
✅ team: sin columnas duplicadas evidentes.
🧽 Nulos limpiados en game: 65698 filas, 55 columnas
🧽 Nulos limpiados en game_summary: 58110 filas, 14 columnas
🧽 Nulos limpiados en other_stats: 28271 filas, 26 columnas
🧽 Nulos limpiados en player: 4831 filas, 5 columnas
🧽 Nulos limpiados en common_player_info: 4171 filas, 33 columnas
🧽 Nulos limpiados en team: 30 filas, 7 columnas
✅ game guardado en: C:\Users\juanl\Downloads\final\csv\game_clean.csv
✅ game_summary guardado en: C:\Users\juanl\Downloads\final\csv\game_summary_clean.csv
✅ other_stats guardado en: C:\Users\juanl\Downloads\final\csv\other_stats_clean.csv
✅ player guardado en: C:\Users\juanl\Downloads\final\csv\player_clean.csv
✅ common_player_info guardado en: C:\Users\juanl\Downloads\final\csv\common_player_info_clean.csv
✅ team guardado en: C:\Users\juanl\Downloads\final\csv\team_clean.csv

📋 Resumen general de los datasets:

,Dataset,Filas,Columnas,Porc_nulos_prom,Numéricas,Texto,Fechas
0,game,65698,54,0.0,45,8,1
1,game_summary,58110,12,0.0,8,3,1
2,other_stats,28271,26,0.0,22,4,0
3,player,4831,5,0.0,2,3,0
4,common_player_info,4171,32,0.0,7,24,1
5,team,30,7,0.0,2,5,0
